# Simulation test for false positive in metagenomics based taxonomic analysis

In [6]:
from Bio import Entrez
import os
import requests

In [13]:
# Set your email
Entrez.email = "xli@chem.fsu.edu"

# List of RefSeq IDs
refseq_ids = [
    "GCF_000204415.1", "GCF_000016525.1", "GCF_000007345.1",
    "GCF_000063445.1", "GCF_002287195.1", "GCF_002310855.1",
    "GCF_000012325.1", "GCF_000967895.1", "GCF_009362255.1",
    "GCF_001653755.1", "GCF_000208865.1", "GCF_030704535.1",
    "GCF_000150955.2", "GCF_000002765.6", "GCF_032362555.1" ## randomly selected 15 species (5 for bacteria, 5 for archaea, 5 for eukaryota)
]
# List of RefSeq IDs
refseq_ids = [
    "GCF_000063445.1", "GCF_002287195.1", "GCF_002310855.1", "GCF_009362255.1","GCF_000150955.2", ## randomly selected 15 species (5 for bacteria, 5 for archaea, 5 for eukaryota)
]
refseq_ids = [
    "GCF_000063445.1",  ## randomly selected 15 species (5 for bacteria, 5 for archaea, 5 for eukaryota)
]


In [8]:
# Output directory
output_dir = '/home/xiangpeng/projects/16S_18S/code_for_github/simulation/'
os.makedirs(output_dir, exist_ok=True)

In [5]:
# Download FASTA files
for refseq_id in refseq_ids:
    print(f"Downloading {refseq_id}...")
    try:
        # Search and fetch the assembly
        handle = Entrez.esearch(db="assembly", term=refseq_id)
        record = Entrez.read(handle)
        handle.close()
        
        if record["IdList"]:
            assembly_id = record["IdList"][0]
            
            # Fetch assembly summary
            handle = Entrez.esummary(db="assembly", id=assembly_id)
            summary = Entrez.read(handle)
            handle.close()
            
            # Get FTP link for FASTA
            ftp_path = summary['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_RefSeq']
            fasta_url = f"{ftp_path}/{os.path.basename(ftp_path)}_genomic.fna.gz"
            
            # Download the FASTA file
            os.system(f"wget -P {output_dir} {fasta_url}")
        else:
            print(f"RefSeq ID {refseq_id} not found.")
    except Exception as e:
        print(f"Error with {refseq_id}: {e}")

--2024-12-20 17:27:29--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/204/415/GCF_000204415.1_ASM20441v1/GCF_000204415.1_ASM20441v1_genomic.fna.gz
           => ‘/home/xiangpeng/projects/16S_18S/simulation/GCF_000204415.1_ASM20441v1_genomic.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... 


KeyboardInterrupt



In [14]:
# Download FASTA files using HTTPS
for refseq_id in refseq_ids:
    print(f"Processing {refseq_id}...")
    try:
        # Search and fetch the assembly
        handle = Entrez.esearch(db="assembly", term=refseq_id)
        record = Entrez.read(handle)
        handle.close()

        if record["IdList"]:
            assembly_id = record["IdList"][0]

            # Fetch assembly summary
            handle = Entrez.esummary(db="assembly", id=assembly_id)
            summary = Entrez.read(handle)
            handle.close()

            # Get HTTPS link for FASTA
            ftp_path = summary['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_RefSeq']
            https_path = ftp_path.replace("ftp://", "https://")
            fasta_url = f"{https_path}/{os.path.basename(ftp_path)}_genomic.fna.gz"
            
            # Download the FASTA file
            response = requests.get(fasta_url, stream=True)
            if response.status_code == 200:
                output_file = os.path.join(output_dir, os.path.basename(fasta_url))
                with open(output_file, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        f.write(chunk)
                print(f"Downloaded: {output_file}")
            else:
                print(f"Failed to download {refseq_id}: {response.status_code}")
        else:
            print(f"RefSeq ID {refseq_id} not found.")
    except Exception as e:
        print(f"Error with {refseq_id}: {e}")


Processing GCF_000063445.1...
Downloaded: /home/xiangpeng/projects/16S_18S/simulation/GCF_000063445.1_ASM6344v1_genomic.fna.gz


```
GCF_000002765.6_GCA_000002765_genomic.fna.gz  GCF_000967895.1_ASM96789v1_genomic.fna.gz
GCF_000007345.1_ASM734v1_genomic.fna.gz       GCF_001653755.1_ASM165375v1_genomic.fna.gz
GCF_000012325.1_ASM1232v1_genomic.fna.gz      GCF_002287195.1_ASM228719v1_genomic.fna.gz
GCF_000016525.1_ASM1652v1_genomic.fna.gz      GCF_002310855.1_ASM231085v1_genomic.fna.gz
GCF_000063445.1_ASM6344v1_genomic.fna.gz      GCF_009362255.1_ASM936225v1_genomic.fna.gz
GCF_000150955.2_ASM15095v2_genomic.fna.gz     GCF_030704535.1_ASM3070453v1_genomic.fna.gz
GCF_000204415.1_ASM20441v1_genomic.fna.gz     GCF_032362555.1_ilAmyTran1.1_genomic.fna.gz
GCF_000208865.1_ASM20886v2_genomic.fna.gz
```

Run the following code in command line to generate FASTA silulation files. 
```
art_illumina -i GCF_000002765.6_GCA_000002765_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000002765.6
art_illumina -i GCF_000967895.1_ASM96789v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000967895.1
art_illumina -i GCF_000007345.1_ASM734v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000007345.1
art_illumina -i GCF_001653755.1_ASM165375v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_001653755.1
art_illumina -i GCF_000012325.1_ASM1232v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000012325.1
art_illumina -i GCF_002287195.1_ASM228719v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_002287195.1
art_illumina -i GCF_000016525.1_ASM1652v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000016525.1
art_illumina -i GCF_002310855.1_ASM231085v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_002310855.1
art_illumina -i GCF_000063445.1_ASM6344v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000063445.1
art_illumina -i GCF_009362255.1_ASM936225v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_009362255.1
art_illumina -i GCF_000150955.2_ASM15095v2_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000150955.2
art_illumina -i GCF_030704535.1_ASM3070453v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_030704535.1
art_illumina -i GCF_000204415.1_ASM20441v1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000204415.1
art_illumina -i GCF_032362555.1_ilAmyTran1.1_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_032362555.1
art_illumina -i GCF_000208865.1_ASM20886v2_genomic.fna.gz -p -l 150 -m 500 -s 50 -c 1000000 -o GCF_000208865.1
